In [ ]:
from fastcore.all import *
from execnb.nbio import *

In [ ]:
path = Path('../intro.ipynb')

In [ ]:
path = Path('../conclusion.ipynb')

In [ ]:
nb = read_nb(path)

In [ ]:
from bs4 import BeautifulSoup
from bs4.formatter import HTMLFormatter

In [ ]:
b = BeautifulSoup('hi <img alt="Natural" caption="artificial" id="neuron" src="images/chapter7.png" width="500"/>')
im = b.img
im['alt']

'Natural'

In [ ]:
def img2md(im):
    r = '!['
    a = im.get('caption', None)
    if a: r+=a
    r+=f']({im["src"]}'
    r+=')'
    d = {}
    a = im.get('width', None)
    if a: d['width'] = a
    a = im.get('id', None)
    if a: d['id'] = a
    a = im.get('alt', None)
    if a: d['fig-alt'] = a
    if d: r+='{' + ' '.join(f'{k}="{v}"' for k,v in d.items()) + '}'
    return r

In [ ]:
class UnsortedAttributes(HTMLFormatter):
    def attributes(self, tag): yield from tag.attrs.items()

def enc(t): return t.encode_contents(formatter=UnsortedAttributes()).decode('utf-8')

In [ ]:
for c in nb.cells:
    if c.cell_type=='markdown' and '<img ' in c.source:
        b = BeautifulSoup(c.source)
        im = b.img
        im.replace_with(img2md(im))
        o = b.body
        if getattr(o,'p',None): o = o.p
        print(enc(o))

![What to do next](images/att_00053.png){width="550" id="do_next" fig-alt="What to do next"}
